In [ ]:
!pip install streamlit

In [2]:
import os

os.chdir('/content/drive/MyDrive/Omdena/Osun Nigeria/App')

!pwd

/content/drive/MyDrive/Omdena/Osun Nigeria/App


In [3]:
from google.colab import files
uploaded = files.upload()

Saving cassava.jpg to cassava (1).jpg


In [4]:
%%writefile app.py

import streamlit as st
from apps import home, crop_disease_detection
from PIL import Image

with st.container():
    proj_title_col, logo_col = st.columns([6,1])

    with proj_title_col:
      st.subheader('Improving Food Security in Nigeria')

    with logo_col:
      logo = Image.open('/content/drive/MyDrive/Omdena/Osun Nigeria/App/cassava.jpg')
      logo = logo.resize((75,75))
      st.image(logo)

PAGES = {
    "Home": home,
    "Crop Disease Detection": crop_disease_detection
}

selection = st.sidebar.radio("Menu", list(PAGES.keys()))
page = PAGES[selection]

page.app()

Overwriting app.py


In [5]:
%%writefile apps/home.py

import streamlit as st

def app():
  st.subheader('Problem Statement')
  st.write("According to the Food and Agriculture Organisation of the United Nations, 2018, \
  approximately 88 % of the farmers in Nigeria engage in \
  agricultural production at a subsistence level, and they lack sustainable \
  farming knowledge and practices. Also, Nigeria is endowed with \
  different climatic conditions and soil quality which leads to lackluster \
  crop production. ")

  st.markdown('##')

  st.subheader('Objectives of the Project')
  st.write('This project is aimed at helping farmers to boost their \
  farm produce and plan their farming system.')

Overwriting apps/home.py


In [6]:
%%writefile apps/crop_disease_detection.py

import streamlit as st
import streamlit.components.v1 as components
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array
from PIL import Image
import utils

TARGET_SIZE = (100, 100)


@st.cache(allow_output_mutation=True)

def load_model():
  """
  Loads the models for maize and rice. 
  """
  maize_model = tf.keras.models.load_model('/content/drive/MyDrive/Omdena/Osun Nigeria/Model/model_maize.h5')
  rice_model = tf.keras.models.load_model('/content/drive/MyDrive/Omdena/Osun Nigeria/Model/model_rice.h5')
  return maize_model, rice_model

def preprocess_img(image):
  """ Preprocess the image in the same way the images for model building were processed. 
  
  image: uploaded image

  Returns: image preprocessed using resnet50's preprocess_input
  """

  img = image.resize(TARGET_SIZE)
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = preprocess_input(img)
  return img

def predict(image, model):
  """ Predicts the disease of the image.

  image: preprocessed image
  model: model for maize or rice leaf disease datasets

  Returns: probability for each class/disease
  """
  predictions = model.predict(image)
  return predictions

def app():

  new_image = ''
  has_pred = False
  has_replaced_image = False
  pred = ''

  #Load the model
  selection = st.selectbox('Crop Dataset', ('Maize', 'Rice'))
  if selection == 'Maize':
    model = load_model()[0]
  else:
    model = load_model()[1]

  st.markdown('#') #For space between the selection box and the next container

  with st.container():
      upload_col, predict_col = st.columns([6,1])
      with upload_col:
        
        #Upload the image
        uploaded_file = st.file_uploader('Choose a file',type=["jpg", 'png'])
        if uploaded_file is not None:
          img = Image.open(uploaded_file)
          img_display = img.resize((700, 500))
          st.image(img_display)

          #Preprocess image
          img = preprocess_img(img)
          new_image = img
          has_replaced_image = True
          
          
      with predict_col:
        #Predict the disease label for the image
          if st.button('Predict'):
            if has_replaced_image:
              predictions = predict(new_image, model)
              pred = np.argmax(predictions)
              has_pred = True
            else:
              st.error('Please upload an image')

  #This container is for the description of the predicted disease of the image            
  with st.container():
    if has_pred:
      with st.expander("Read disease description"):
        disease = utils.get_class(selection, pred)
        st.title(disease)
        disease_desc = utils.get_description(disease)
        st.markdown(disease_desc, unsafe_allow_html=True)

Overwriting apps/crop_disease_detection.py


In [7]:
%%writefile utils.py

def get_class(crop, idx):
  """ Gets the disease label for the predicted image.

  crop: crop type - maize or rice
  idx: column index of the class with the highest probability in the predictions

  Returns: disease label

  """
  if crop == 'Maize':
    CLASSES = {
        0: 'Blight',
        1: 'Common Rust',
        2: 'Gray Leaf Spot',
        3: 'Healthy'
    }
  else:
    CLASSES = {
        0: 'Bacterial Leaf Blight',
        1: 'Brown Spot',
        2: 'Leaf Smut'
    }

  return CLASSES[idx]

def get_description(disease):
  """ Gets the disease description

    disease: predicted disease for the image
  """
  maize_blight = f"""
  <p> Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
  Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. 
  Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. 
  Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
  </p>
  """

  maize_common_rust = f"""
  <p> Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. 
  Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.
  </p>
  """
  maize_gray_leaf_spot = f"""
  <p> Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. 
  Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.
  </p>
  """

  maize_healthy = f"""
  <p> Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. 
  Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.
  </p>
  """

  rice_bacterial_leaf_blight = f"""
  <p> Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. 
  Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.
  </p>
  """

  rice_brown_spot = f"""
  <p> Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. 
  Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.
  </p>
  """

  rice_leaf_smut = f"""
  <p> Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. 
  Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.
  </p>
  """

  dict_description = {
      'Blight': maize_blight,
      'Common Rust': maize_common_rust
      'Gray Leaf Spot': maize_gray_leaf_spot,
      'Healthy': maize_healthy,
      'Bacterial Leaf Blight': rice_bacterial_leaf_blight,
      'Brown Spot': rice_brown_spot,
      'Leaf Smut': rice_leaf_smut
  }

  return dict_description[disease]


Overwriting utils.py


In [8]:
!streamlit run app.py & npx localtunnel --port 8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.10.218:8501

npx: installed 22 in 3.068s
your url is: https://empty-catfish-15.loca.lt
2021-10-23 14:37:41.855062: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-23 14:37:41.855108: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bbec008d8e1a): /proc/driver/nvidia/version does not exist
2021-10-23 14:37:56.819025: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
  Stopping...
^C
